# Feature Engineering

La previsione dell'indice giornaliero futuro di rischio incidenti si baserà su tre tipi di features:
- <font color="dodgerblue">**features spaziali**</font>: features statiche che fanno riferimento alla geografia urbana della zona specifica e permettono di discriminare la pericolosità di un punto in città rispetto ad un altro. Variano abbastanza lentamente nel tempo da essere considerate statiche.
- <font color="dodgerblue">**features temporali**</font>: features dinamiche che cambiano in base al giorno ma sono abbastanza uniformi a livello spaziale; permettono di discrimare la pericolosità di un giorno rispetto ad un altro. Es. meteo, giorno della settimana...
- <font color="dodgerblue">**features miste**</font>: features che variano in base al giorno ma possono essere piuttosto eterogenee anche a livello geografico. Es. traffico o eventi in specifiche zone della città, storico di incidenti nella zona.

La combinazione di questi 3 tipi di features permette di avere variabili variegate in input a un modello di Machine Learning e riuscire ad effettuare una previsione più accurata sia nel tempo che nello spazio.

In [ ]:
import os
os.environ["USE_PYGEOS"] = "0"

import pandas as pd
import numpy as np
import osmnx as ox
import folium
import geojson
import geopandas as gpd
import math
import matplotlib.pyplot as plt
import re
import seaborn as sns

from pointpats import centrography
from pyproj import Transformer
from shapely import wkt, buffer
from shapely.geometry import Polygon, Point
from shapely.prepared import prep
from sklearn.cluster import DBSCAN
from google.cloud import storage as gcs

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../serviceaccount-piattaformapa-test.json"

pd.options.display.max_columns = 200

In [ ]:
hyperparams = {
    "city": "Milan",
    "grid": 
        {
            "hexagon_height": 1000
        },
    "clustering":
        {
            "dbscan": 
                {
                    "min_samples": 4,
                    "eps": 30,
                },
            "shared_grid": "cluster_hull"   # choose between "cluster_hull" or "enc_circle"
        },
    "centrality":
        {
            "percentile": 0.9,
        }
}

In [ ]:
bucket_roads = "bucket-cityroads-data-test"
bucket_boundaries = "bucket-city-boundaries-data-test"

nil_file = "nil.geojson"
municipi_file = "municipi.geojson"

city = "Milan"

client = gcs.Client()

# city boundaries
bucket_boundaries = client.get_bucket(bucket_boundaries)

nil = bucket_boundaries.blob(nil_file)
municipi = bucket_boundaries.blob(municipi_file)

with nil.open() as f:
    nil_dict = geojson.load(f)
nil_df = gpd.read_file(nil.open())
nil_features = nil_dict["features"]

with municipi.open() as f:
    municipi_dict = geojson.load(f)
municipi_df = gpd.read_file(municipi.open())
municipi_features = municipi_dict["features"]

## Divisione della città in zone

Come anticipato, la predizione dell'indice di rischio sarà divisa per "zone", in modo da discriminare i diversi punti della città, potenzialmente molto diversi a livello di rischio incidenti.

La divisione per zone non può avvenire per municipi o quartieri, trattandosi di divisioni territoriali irregolari e di diverse dimensioni.<br>
Per ovviare a tale problema, si divide la città in **griglie esagonali** (ref link: [Why hexagons?](https://pro.arcgis.com/en/pro-app/latest/tool-reference/spatial-statistics/h-whyhexagons.htm)). La dimesione delle griglie è fissa ed è un iperparametro del problema ma va scelta secondo i seguenti ragionamenti:
- griglie troppo grandi: dimensione ridotta del dataset, predizioni troppo grezze e incapacità di dividere correttamente le diverse zone della città.
- griglie troppo piccole: incapacità di estrarre pattern geografici e temporali significativi dovuta a un'osservazione troppo puntuale (rischio di ottenere features uguali per tutte le griglie).

La dimensione ottima è quella che consente di avere una griglia abbastanza fine ma che riesca a dividere intelligentemente zone diverse.

Ad ogni griglia viene poi associato il/i corrispondete/i nil e municipi con la corrispondete percentuale di copertura.

In [ ]:
city_gdf = ox.geocode_to_gdf({"city": city})
city_geom = city_gdf.geometry[0]

# boundaries of the city (coordinates)
minx, miny, maxx, maxy = city_geom.bounds

trans_direct = Transformer.from_crs("EPSG:4326", "EPSG:32632", always_xy=True)
trans_inverse = Transformer.from_crs("EPSG:32632", "EPSG:4326", always_xy=True)

# Temporarly transform coordinates so that we can work directly with distances
minx, miny = (int(res) for res in trans_direct.transform(minx, miny))
maxx, maxy = (int(res) for res in trans_direct.transform(maxx, maxy))

# Define height and radius of each hexagon
h = hyperparams["grid"]["hexagon_height"]
r = int(h / math.sqrt(3))


# Initialize geo dictionary for grids (it can be useful to save as .geojson)
hex_geo_dict = {
    "type": "FeatureCollection",
    "name": "hex_grids_{}".format(city),
    "features": []
}


def hex_grid(h, r):

    # Assign to each grid an id trough
    cont = 0

    # Iterate along x and y axis
    for x in range(minx, maxx, h):
        for y in range(miny, maxy, int(h * h / r / 2)):
            # hexagon geometry
            hexagon = Polygon([
                        [*trans_inverse.transform(x, y + r)],
                        [*trans_inverse.transform(x + h / 2, y + r / 2)],
                        [*trans_inverse.transform(x + h / 2, y - r / 2)],
                        [*trans_inverse.transform(x, y - r)],
                        [*trans_inverse.transform(x - h / 2, y - r / 2)],
                        [*trans_inverse.transform(x - h / 2, y + r / 2)],
                        [*trans_inverse.transform(x, y + r)],
                    ])

            # List of intersected nils and municipi for each grid
            nil_list = []
            municipi_list = []

            # List of weights for each intersected nils and municipi for each grid
            nil_perc = []
            municipi_perc = []

            for nil in nil_features:
                prepared_geom = prep(Polygon(nil["geometry"]["coordinates"][0]))
                # Only save if the grid intersects any nil
                if prepared_geom.intersects(hexagon):
                    # Compute percentage of intersection area
                    perc_area = round(
                        hexagon.intersection(Polygon(nil["geometry"]["coordinates"][0])).area/hexagon.area,
                        3,
                    )
                    # Save nil infos
                    nil_list.append(nil["properties"]["NIL"])
                    nil_perc.append(perc_area)

            # Same for municipi
            for mun in municipi_features:
                prepared_geom = prep(Polygon(mun["geometry"]["coordinates"][0]))
                if prepared_geom.intersects(hexagon):
                    perc_area = round(
                        hexagon.intersection(Polygon(mun["geometry"]["coordinates"][0])).area/hexagon.area,
                        3,
                    )
                    municipi_list.append(mun["id"])
                    municipi_perc.append(perc_area)

            # Grid is saved only if its intersection is non empty
            if (len(municipi_list) > 0) or (len(nil_list) > 0):
                final_dict = {
                    "type": "Feature",
                    "id_grid": cont,
                    "nil_list": nil_list,
                    "nil_weights": nil_perc,
                    "municipi_list": municipi_list,
                    "municipi_weights": municipi_perc,
                    "geometry": hexagon,
                }

                hex_geo_dict["features"].append(final_dict)
                cont += 1
            
            # offset to be added to avoid overlapping of hexagons
            i = math.floor((maxy - y) / r)
            offset = -h / 2 if i % 2 == 0 else h / 2

            x += offset

    print("Drawn {} hexagon grids with height {} meters".format(cont, h))

hex_grid(h, r)

hex_df = gpd.GeoDataFrame(hex_geo_dict["features"], geometry="geometry", crs=4326).drop(columns="type")
hex_df.sample(3)

## Spatial Feature Engineering (dati da OpenStreetMap)

La prima fase di feature engineering consiste nell'estrazione, processing, aggregazione di dati spaziali in ambito urbano e conseguente associazione spaziale con ogni griglia.

Vengono estratte informazioni che sono notoriamente più rilevanti e correlate con il rischio di incidenti:
- <font color="dodgerblue">**semafori**</font>
- <font color="dodgerblue">**precedenze e stop**</font>
- <font color="dodgerblue">**rotatorie**</font>
- <font color="dodgerblue">**rotaie tram**</font>
- <font color="dodgerblue">**manto stradale**</font>

Tali informazioni vengono poi processate per creare delle features rilevanti per ogni griglia.

Viene inoltre estratto l'intero <font color="dodgerblue">**reticolo stradale**</font> della città di interesse.<br>
Quest'ultima estrazione permette di rappresentare le strade di una città come un **_grafo_** (gli archi sono i segmenti di strada e i nodi le intersezioni tra di esse), ed estrarre dei pattern interessanti dal punto di vista della viabilità:
- <font color="dodgerblue">**strade più importanti**</font>: l'importanza di una strada può essere associate ad alcune metriche di centralità di nodi in un grafo (dettagli in seguito).

L'informazione sull'importanza delle strade viene poi processata per creare delle features per ogni griglia.

##### Graph analysis

Le strade di una città possono essere rappresentate come un grafo, sul quale possono essere applicati degli algoritmi per capire quali archi (strade) sono le più importanti e critiche all'interno del dominio della città, secondo vari metriche:
- **Betweeneess centrality**: misura quante volte un nodo risiede nel percorso più veloce tra due nodi. I nodi, e le strade associate, che hanno un alto punteggio, hanno un'influenza più alta negli spostamenti all'interno del grafo.
- **Closeness centrality**: misura la distanza media di ogni nodo dagli altri nodi. Un punteggio alto identifica quindi i nodi (e strade) più "centrali" nella città.
- **Node degree**: il numero di segmenti stradali che entrano ed escono da ogni nodo. Le strade identificate da due nodi con alto punteggio, sono quelle nelle quali affluiscono più strade.

In [ ]:
bucket_roads = "bucket-cityroads-data-test"

csv_roads_iter = client.list_blobs(bucket_roads, prefix="Milan")


# city items
df_dict = {}

for blob in csv_roads_iter:
    df_path = "gs://{}/{}".format(bucket_roads, blob.name)
    df_name = re.split("\/|\.", blob.name)[1]
    df = pd.read_csv(df_path)
    df["geometry"] = df["geometry"].apply(wkt.loads)
    if df_name != "roads_graph_data":
        df["type"] = df_name
    df_dict[df_name] = gpd.GeoDataFrame(df, crs="epsg:4326", geometry="geometry")

percentile = hyperparams["centrality"]["percentile"]
perc_dict = {}

col_perc_exclusion = [
    "name",
    "u",
    "v",
    "oneway",
    "lanes",
    "highway",
    "geometry",
    "junction",
    "point_geometry_u",
    "point_geometry_v",
]

col_percentile_list = [
    col for col in df_dict["roads_graph_data"] if col not in col_perc_exclusion
]

for col in col_percentile_list:
    perc_dict[col] = df_dict["roads_graph_data"][col].quantile(percentile)

##### Plots

In [ ]:
def plot_centrality(df, col, color, m, only_main):
    if only_main:
        df_filt = df[
            (df["avg_sub_" + col] > perc_dict["avg_sub_" + col])
        ]
        df_filt\
            .explore(name=col + " main edges", m=m, color=color, style_kwds = {"weight": 2})
    else:
        df.explore(
            name=col + " edges",
            m=m,
            column="avg_sub_" + col,
            cmap="seismic",
            style_kwds = {"weight": 2}
        )

def plot_items(m):
    cs = iter(["red", "blue", "orange", "green", "eggshell", "black"])
    for key, df in df_dict.items():
        if key in ["roads_graph_data", "parcometri", "colonnine"]:
            continue
        df.explore(
            name=key,
            column="surface_mapped" if key == "manto_stradale" else None,
            m=m,
            color=None if key == "manto_stradale" else next(cs),
            cmap=["red", "blue"] if key == "manto_stradale" else None,
        )

def plot_grid_nil_municipi(nil_and_municipi):
    city_map = city_gdf.explore(
    name="Milan urban perimeter",
    # tiles="OpenStreetMap Mapnik",
    style_kwds = {
        "fill": False,
        "color": "black",
        "weight": 3
    }
    )
    hex_df.explore(
    name="hex grid",
    m=city_map,
    style_kwds = {
        "fill": False,
        "color": "red",
        "fillColor": "white",
        "fillOpacity": 0.1,
        "weight": 2,
    },
)
    if nil_and_municipi:
        gpd.GeoDataFrame.from_features(nil_dict["features"]).explore(
            name="nil",
            m=city_map,
            style_kwds = {
                "fill": False,
                "color": "blue",
                "fillColor": "white",
                "fillOpacity": 0.1,
                "weight": 1.5,
            },
        )

        gpd.GeoDataFrame.from_features(municipi_dict["features"]).explore(
            name="municipi",
            m=city_map,
            style_kwds = {
                "fill": False,
                "color": "green",
                "fillColor": "white",
                "fillOpacity": 0.1,
                "weight": 1.5,
            },
        )
    return city_map

In [ ]:
city_map_items = plot_grid_nil_municipi(nil_and_municipi=True)

plot_items(city_map_items)

folium.LayerControl(collapsed=False).add_to(city_map_items)

city_map_items

In [ ]:
centrality_metrics = {
    "closeness": "red",
    "betweenness": "blue",
    # "pagerank": "orange",
    # "degree": "green",
    "adjacent_roads": "dodgerblue",
}
only_main = True


city_map_centrality = plot_grid_nil_municipi(nil_and_municipi=True)

for cen, color in centrality_metrics.items():
    plot_centrality(
        df=df_dict["roads_graph_data"],
        col=cen,
        color=color,
        m=city_map_centrality,
        only_main=only_main,
    )

folium.LayerControl(collapsed=False).add_to(city_map_centrality)

city_map_centrality

## 1. Identificazione punti critici tramite clustering

Un altro modo per identificare i punti più caldi della viabilità urbana è identificare gli spots nei quali si concentra un alto numero di semafori. Questi punti potrebbero essere soggetti a un maggior numero di incidenti proprio per la presenza concentrata di elementi di controllo del traffico.

Per identificare questi punti critici, si può utilizzare un algoritmo di clustering come DBSCAN, che tramite il tuning alcuni parametri, riesce automaticamente a separare punti fortemente clusterizzati da quelli più isolati.

Il convex hull di ogni cluster viene poi intersecato geograficamente con le griglie.<br>
Alcuni cluster possono cadere tra due o più griglie: in questo caso il cluster viene assegnato ad ogni griglia, ma viene anche riportato il numero effettivo di elementi nel cluster appartenente alla griglia.

Le features estratte per ogni griglia sono quindi le seguenti:
- <font color="dodgerblue">**numero di cluster**</font>
- <font color="dodgerblue">**numero di elementi del cluster più grande**</font>
- <font color="dodgerblue">**numero medio di elementi dei cluster**</font>
- <font color="dodgerblue">**percentuale di elementi in cluster condivisi che appartengono alla griglia**</font>

## 2. Numero di elementi e metri totali di categorie di strada

Tramite le informazioni citate sopra, possono essere create delle features per ogni griglia:
- <font color="dodgerblue">**Numero di semafori**</font>
- <font color="dodgerblue">**Numero di stop**</font>
- <font color="dodgerblue">**Numero di precedenze**</font>
- <font color="dodgerblue">**Metri totali di rotatorie**</font>
- <font color="dodgerblue">**Metri totali di rotaie tram**</font>
- <font color="dodgerblue">**Percentuale di strade asfaltate**</font>
- <font color="dodgerblue">**lunghezza totale strade**</font>
- <font color="dodgerblue">**lunghezza totale per tipologia di strada**</font>: residenziale, primaria, secondaria, terziaria.

L'analisi sul grafo permette invece di creare altri tipi di features:
- <font color="dodgerblue">**Punteggio medio di centralità delle strade**</font>: Per ognuno dei punteggi citati sopra.
- <font color="dodgerblue">**metri di strade con punteggio di centralità sopra il 90esimo percentile**</font>: Per ognuno dei punteggi citati sopra.


**-------------------------------------------------------------------------------------------------------------------------------------------------------------**

### Clustering

In [ ]:
def smallest_enclosing_circle(df):
    center = Point(centrography.mean_center(
        df[["x", "y"]].values,
    ))

    max_dist = 0
    for p in df["geometry"]:
        dist = p.distance(center)
        if dist > max_dist:
            max_dist = dist
    return buffer(center, max_dist)

def cluster_items(df, item):
    clusterer = DBSCAN(
        eps=hyperparams["clustering"]["dbscan"]["eps"],
        min_samples=hyperparams["clustering"]["dbscan"]["min_samples"],
    )

    df["x"] = df.apply(lambda x: x["geometry"].coords[0][0], axis=1)
    df["y"] = df.apply(lambda x: x["geometry"].coords[0][1], axis=1)

    df["x_tr"], df["y_tr"] = trans_direct.transform(df["x"].values, df["y"].values)

    cluster_label = clusterer.fit(np.column_stack([df["x_tr"], df["y_tr"]])).labels_
    df["cluster_label"] = cluster_label
    df["label_type"] = "cluster"
    df.loc[df.cluster_label == -1, "label_type"] = "noise"

    hull_df = df[["geometry", "cluster_label"]].dissolve("cluster_label").convex_hull.to_frame().reset_index()

    df = df.merge(
        hull_df,
        on="cluster_label",
    )
    df.rename(columns={0: "cluster_hull"}, inplace=True)
    
    enc_circle = df.groupby("cluster_label")[["geometry", "x", "y"]].apply(
        smallest_enclosing_circle
    ).to_frame().reset_index().rename(columns={0: "enc_circle"})

    df = df.merge(enc_circle)
    df.loc[df.cluster_label == -1, "cluster_hull"] = np.nan
    df.loc[df.cluster_label == -1, "enc_circle"] = np.nan
    df["n_samples_clust"] = df.groupby("cluster_label")["osmid"].transform("count")
    
    return df.drop(columns=["x", "y", "x_tr", "y_tr"])

for item in ["semaforo"]:
    df_dict[item + "_cluster"] = cluster_items(df_dict[item], item)
    clust_df = df_dict[item + "_cluster"]

In [ ]:
col_to_intersect = hyperparams["clustering"]["shared_grid"]

# Spatial merge grid with cluster hulls
merge_hull_df = clust_df\
        [clust_df.cluster_label != -1]\
        [["cluster_label", col_to_intersect, "n_samples_clust"]]\
        .set_geometry(col_to_intersect, crs=4326)

merged_grid_clust = (hex_df
        .sjoin(
            merge_hull_df,
            how="left",
            predicate="intersects",
            lsuffix="grid",
            rsuffix="semaforo",
        )
    )
merged_grid_clust = merged_grid_clust.drop("index_semaforo", axis=1)


# Spatial merge single traffic lights with grids
merge_tl_df = clust_df\
        [clust_df.cluster_label != -1]\
        [["osmid", "geometry", "cluster_label"]]

merged_grid_clust = (merged_grid_clust
        .sjoin(
            merge_tl_df,
            how="left",
            predicate="contains",
        )
    )

merged_grid_clust = merged_grid_clust[
    (merged_grid_clust.cluster_label_left == merged_grid_clust.cluster_label_right)
    |
    (merged_grid_clust.cluster_label_left.isna())
    ]

merged_grid_clust = merged_grid_clust.drop("index_right", axis=1)

# Compute how many traffic lights for cluster for each grid
# actually belong to the grid
def compute_perc_actual_tf(df):
    # sum of all clustered tf in the grid
    n_tf_clusters = df.drop_duplicates("cluster_label_right").n_samples_clust.sum()
    n_tf_actual = df.osmid.nunique()
    return round(n_tf_actual/n_tf_clusters, 3)
    

perc_actual_tf = merged_grid_clust.groupby(["id_grid"])[["osmid", "n_samples_clust", "cluster_label_right"]]\
    .apply(compute_perc_actual_tf)\
    .to_frame().reset_index()

merged_grid_clust = merged_grid_clust\
    .merge(perc_actual_tf, how="left")\
    .rename(columns={0: "perc_clust_points_actual"})\
    .drop(columns="cluster_label_right")\
    .rename(columns={"cluster_label_left": "cluster_label"})\
    .drop(columns="osmid")\
    .drop_duplicates(["id_grid", "cluster_label"])


tmp_groupby = merged_grid_clust.groupby("id_grid").agg(
    {
        "cluster_label": lambda x: x.nunique(),
        "n_samples_clust": ["max", "mean"]
    }
)
tmp_groupby.columns = tmp_groupby.columns.droplevel()
tmp_groupby = tmp_groupby.reset_index()

merged_grid_clust = merged_grid_clust\
    .merge(tmp_groupby, on="id_grid", how="left")
merged_grid_clust.rename(columns={
    "<lambda>": "n_clusters",
    "max": "max_cluster_size",
    "mean": "mean_cluster_size"
}, inplace=1)
merged_grid_clust.fillna(0, inplace=True)

merged_grid_clust["max_cluster_size"] = merged_grid_clust["max_cluster_size"].apply(lambda x: int(x))
merged_grid_clust["mean_cluster_size"] = merged_grid_clust["mean_cluster_size"].apply(lambda x: round(x, 3))

merged_grid_clust = merged_grid_clust\
    .drop_duplicates("id_grid")\
    .drop(columns=["cluster_label", "n_samples_clust"])

##### Cluster plot

In [ ]:
def plot_clusters(city_map, item, color_point):
    clust_df[["geometry", "cluster_label"]][clust_df.cluster_label != -1]\
        .explore(
            name=item + " clusters",
            m=city_map,
            color=color_point,
        )

    clust_df[["geometry", "cluster_label"]][clust_df.cluster_label == -1]\
        .explore(
            name=item + " noise",
            m=city_map,
            color="black",
        )

    clust_df[clust_df.cluster_label != -1][["cluster_hull"]]\
            .set_geometry("cluster_hull", crs=4326).explore(
                name=item + " cluster hull",
                m=city_map,
                style_kwds = {
                    "fill": False,
                    "color": "green",
                    "weight": 1.5
                }
            )
    return city_map


cs =iter(["dodgerblue"])
city_map = plot_grid_nil_municipi(nil_and_municipi=False)

for item in ["semaforo"]:
    city_map = plot_clusters(city_map, item, next(cs))

popup1 = folium.LatLngPopup()
city_map.add_child(popup1)
folium.LayerControl(collapsed=False).add_to(city_map)

city_map

**-------------------------------------------------------------------------------------------------------------------------------------------------------------**

##### Conteggio elementi

In [ ]:
def merge_grid_with_items(
        input_df,
        item,
        linestring=False,
        surface=False,
        type=False,
        score=False,
        centr_metrics=[]
    ):
    cols_to_include = ["osmid", "geometry"]
    if surface:
        cols_to_include = ["osmid", "geometry", "surface_mapped"]
    if type:
        cols_to_include = ["osmid", "geometry", "highway"]
    if score:
        cols_to_include = ["osmid", "geometry", *["avg_sub_" + c  for c in centr_metrics]]
    item_df = df_dict[item][cols_to_include]
    item_df["osmid"] == item_df["osmid"].astype(str)
    item_df = item_df.rename(
        columns={
            "geometry": "{}_geometry".format(item),
            "osmid": "{}_osmid".format(item),
            }
        )
    item_df.set_geometry("{}_geometry".format(item), crs=4326, inplace=1)
    if linestring:
        item_df["geometry_length"] = item_df["{}_geometry".format(item)]
    else:
        input_df = input_df[["id_grid", "geometry"]]

    merged = (input_df
        .sjoin(
            item_df,
            how="left",
            predicate="intersects",
            rsuffix = "{}".format(item),
        )
        .drop(
            columns="index_{}".format(item)
        )
    )
    if linestring:
        merged.set_geometry("geometry_length", crs=4326, inplace=1)
    return merged


def count_item_points_grid(input_df, item):
    merged = merge_grid_with_items(input_df, item)
    merged = merged\
        .groupby("id_grid")["{}_osmid".format(item)]\
        .count()\
        .to_frame()\
        .reset_index()\
        .rename(columns={"{}_osmid".format(item): "n_{}".format(item)})
    merged = input_df\
        .merge(merged, how="left")
    # merged.set_geometry("geometry", inplace=1)
    return merged


feature_df = merged_grid_clust
feature_df = gpd.GeoDataFrame(feature_df, geometry="geometry", crs="EPSG:4326")

list_item_count = [
    "precedenza",
    "semaforo",
    "stop"
]
for item in list_item_count:
    print(item)
    feature_df = count_item_points_grid(feature_df, item)

##### Calcolo lunghezze

In [ ]:
def compute_length(input_df, item):
    merged = merge_grid_with_items(input_df, item, linestring=True)
    merged = merged.dissolve("id_grid", as_index=False)
    merged["length_{}".format(item)] = round(merged["geometry_length"].to_crs(32632).length, 3)
    merged.drop(columns=[
        "{}_osmid".format(item),
        "geometry_length",
    ], inplace=True)

    merged.set_geometry("geometry", crs=4326, inplace=True)

    return merged

list_item_length = [
    "rotatoria",
    "rotaie_tram"
]

df = df_dict["rotatoria"]
df = df[~(df["highway"].str.contains("motorway"))]
df_dict["rotatoria"] = gpd.GeoDataFrame(df, geometry="geometry", crs=4236)

for item in list_item_length:
    print(item)
    feature_df = compute_length(feature_df, item)

##### Percentuale asfalto

In [ ]:
def compute_perc_asphalt(input_df, item):
    merged = merge_grid_with_items(input_df, item, linestring=True, surface=True)
    
    asphalt_length = merged[merged.surface_mapped == "asphalt"].dissolve("id_grid", as_index=False)
    asphalt_length["asph_length_{}".format(item)] = round(asphalt_length["geometry_length"].to_crs(32632).length, 3)
    asphalt_length = asphalt_length[["id_grid", "asph_length_{}".format(item)]]

    total_length = merged.dissolve("id_grid", as_index=False)
    total_length["tot_length_{}".format(item)] = round(total_length["geometry_length"].to_crs(32632).length, 3)
    total_length = total_length[["id_grid", "tot_length_{}".format(item)]]

    tmp = total_length\
        .merge(asphalt_length, how="left").fillna(0)
    tmp["perc_asphalt"] = round(tmp["asph_length_manto_stradale"]/tmp["tot_length_manto_stradale"], 3)
    tmp.drop(
        columns=
        [
            "asph_length_manto_stradale",
            "tot_length_manto_stradale",
        ],
        inplace=True
    )
    merged = merged.drop(
        columns=
        [
            "surface_mapped",
            "{}_osmid".format(item),
            "geometry_length"
        ]
        ).drop_duplicates("id_grid")

    merged = merged\
        .merge(tmp, how="left")
    merged = merged.set_geometry("geometry", crs=4326)
    return merged

df = df_dict["manto_stradale"]
df = df[~(df["highway"].str.contains("motorway"))]
df_dict["manto_stradale"] = gpd.GeoDataFrame(df, geometry="geometry", crs=4236)

feature_df = compute_perc_asphalt(feature_df, "manto_stradale")

##### Totale lunghezza strade per tipo di strada

In [ ]:
df = df_dict["roads_graph_data"]
df = df[~(df["highway"].str.contains("motorway"))]
df["osmid"] = df["u"].astype(str) + df["v"].astype(str)
df_dict["roads_graph_data"] = gpd.GeoDataFrame(df, geometry="geometry", crs=4236)

feature_df = compute_length(
    gpd.GeoDataFrame(feature_df, geometry="geometry", crs=4326),
    "roads_graph_data"
)

def compute_type_road_length(input_df, item):
    merged = merge_grid_with_items(input_df, item, linestring=True, type=True)

    for t in [
        "residential",
        "primary",
        "secondary",
        "tertiary",
        "living_street",
        "unclassified",
    ]:
    
        type_length = merged[merged.highway.str.contains(t, na=False)].dissolve(["id_grid", "highway"], as_index=False)
        type_length["{}_road_length".format(t)] = round(type_length["geometry_length"].to_crs(32632).length, 3)
        type_length = type_length[["id_grid", "{}_road_length".format(t)]]

        merged = merged\
            .merge(type_length, how="left")
        
    merged = merged.drop(
        columns=
        [
            "highway",
            "{}_osmid".format(item),
            "geometry_length"
        ]
        ).drop_duplicates("id_grid")
    
    merged.set_geometry("geometry", crs=4326, inplace=True)
    return merged

feature_df = compute_type_road_length(feature_df, "roads_graph_data")

##### Metriche di centralità

In [ ]:
def count_avg_centrality_grid(input_df, item, centr_metrics):
    merged = merge_grid_with_items(input_df, item, centr_metrics=centr_metrics, score=True)
    cols = ["avg_sub_" + c for c in centr_metrics]
    merged = merged\
        .groupby("id_grid")[cols]\
        .mean()\
        .reset_index()
    merged = input_df\
        .merge(merged, how="left")
    merged = merged.rename(columns={
        "avg_sub_" + c: "avg_grid_" + c for c in centr_metrics
    })
    for c in centr_metrics: merged["avg_grid_" + c] = round(merged["avg_grid_" + c], 3)
    merged.set_geometry("geometry", crs=4326, inplace=True)
    return merged

centr_metrics = ["betweenness", "closeness", "adjacent_roads"]

feature_df = count_avg_centrality_grid(feature_df, "roads_graph_data", centr_metrics)

def length_percentile_centrality_grid(input_df, item, centr_metrics):
    merged = merge_grid_with_items(input_df, item, linestring=True, centr_metrics=centr_metrics, score=True)
    for cen in centr_metrics:
    
        cen_length = merged[merged["avg_sub_" + cen] >= perc_dict["avg_sub_" + cen]].dissolve("id_grid", as_index=False)
        cen_length["{}_perc_{}_length".format(int(percentile*100), cen)] = round(cen_length["geometry_length"].to_crs(32632).length, 3)
        cen_length = cen_length[["id_grid", "{}_perc_{}_length".format(int(percentile*100), cen)]]

        merged = merged.drop(columns="avg_sub_" + cen)\
            .merge(cen_length, how="left")
        
    merged = merged.drop(
        columns=
        [
            "{}_osmid".format(item),
            "geometry_length",
        ]
        ).drop_duplicates("id_grid")
    
    merged.set_geometry("geometry", crs=4326, inplace=True)
    return merged

feature_df = length_percentile_centrality_grid(feature_df, "roads_graph_data", centr_metrics)
feature_df = feature_df.fillna(0)

### Correlazione tra features

In [ ]:
sns.set_theme(style="white")

def corr_matrix(df):
    

    df = df.corr()

    plt.figure(figsize=(16, 6))
    # define the mask to set the values in the upper triangle to True
    mask = np.triu(np.ones_like(df, dtype=bool))
    heatmap = sns.heatmap(
        df,
        mask=mask,
        vmin=df.min().min(),
        vmax=df.max().max(),
        annot=True,
        cmap="coolwarm",
        annot_kws={"size": 9},
        linewidths=.2,
        linecolor="white"
    )
    heatmap.set_xticklabels(heatmap.get_xticklabels(), rotation=45, horizontalalignment="right")
    # heatmap.xaxis.tick_bottom()

    heatmap.set_title("Feature correlation", fontdict={"fontsize":18}, pad=16)


corr_matrix(feature_df.iloc[:, 6:])